In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-t1dj_srd
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-t1dj_srd
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit e523ca0519288b3117f014e19163fe4c514d20ea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-ggc0pffb/elegantrl_021ff78a30904c1fac209d8c627fc83c
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-ggc0pffb/elegantrl_021ff78a30904c1fac209d8c627fc83c
  Resolved https://github.com/AI4Finance-Foundation/ElegantRL.git to commit 47831d796eac39cd14dc75bad1f21db1d1d6ad66
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.config import INDICATORS

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


# 1. Data Wrangle

In [ ]:
TRAIN_START_DATE = '2020-01-01'
TRAIN_END_DATE = '2020-12-31'
TRADE_START_DATE = '2021-09-01'
TRADE_END_DATE = '2021-12-31'

symbols = [
    'BTC-USD',
    'ETH-USD',
    'BNB-USD',
    'ADA-USD',
    'XRP-USD',
    'SOL-USD',
    'BCH-USD'
]

In [ ]:
#Future improvement would be to use Alpaca/Binance

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader

df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = symbols).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (5010, 8)


In [ ]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2020-01-01,0.032832,0.033813,0.032704,0.033458,22948374,ADA-USD,2
1,2020-01-01,204.671295,208.077515,203.205154,204.397537,1456113692,BCH-USD,2
2,2020-01-01,13.730962,13.873946,13.654942,13.689083,172980718,BNB-USD,2
3,2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,BTC-USD,2
4,2020-01-01,129.630661,132.835358,129.198288,130.802002,7935230330,ETH-USD,2


## Preprocess Data

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature=False)

processed = fe.preprocess_data(df_raw)


[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (503, 8)
Successfully added vix


Successfully added turbulence index


In [ ]:
processed.sample(5)

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
341,2020-03-24,0.157574,0.163493,0.156824,0.162255,2155981658,XRP-USD,1,-0.022563,0.249352,0.104247,40.831615,-64.161300,34.569071,0.197733,0.234553,61.669998,0.000000
314,2020-03-18,10.036181,10.236060,9.760224,10.119137,264539835,BNB-USD,2,-2.947726,24.890159,6.771835,35.018705,-138.111949,71.633373,17.751043,19.060886,76.449997,0.000000
2939,2021-12-09,0.862143,0.930541,0.835129,0.861272,4555815213,XRP-USD,3,-0.069225,1.135910,0.772890,40.931107,-93.087514,42.961376,1.018722,1.072581,21.580000,2.298725
266,2020-03-06,20.827026,21.701672,20.783857,21.288389,449180749,BNB-USD,4,-0.321443,24.521149,17.662066,54.757452,-29.087014,0.535759,22.126328,19.631421,41.939999,0.000000
641,2020-06-04,0.203719,0.206494,0.201864,0.204982,1368430496,XRP-USD,3,0.000560,0.209226,0.192897,51.247052,19.353707,19.893121,0.202849,0.200716,25.809999,0.000000


In [ ]:
import itertools

list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).
                 astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)


In [ ]:
processed_full.sample(5)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
3266,2021-06-29,BNB-USD,289.919128,315.439331,289.221161,300.211548,1.946428e+09,1.0,-26.840167,393.434726,250.594171,44.430392,-78.820144,30.184750,341.639229,420.444527,16.020000,1.143986
808,2020-05-15,ETH-USD,202.955399,203.566391,193.755676,195.622665,1.660234e+10,4.0,4.147015,222.633726,184.914356,52.589388,10.436115,2.211631,196.174037,169.597588,31.889999,0.000000
877,2020-05-27,BCH-USD,227.278503,234.030792,226.866913,233.520950,3.107897e+09,2.0,-3.463298,259.386123,217.625276,47.553633,-86.468208,10.258313,242.714718,238.574233,27.620001,0.000000
953,2020-06-08,XRP-USD,0.203426,0.204178,0.201667,0.203771,1.042709e+09,0.0,0.000586,0.209410,0.193108,50.781265,50.306192,11.133166,0.200969,0.201178,25.809999,0.000000
1882,2020-11-10,ETH-USD,444.166382,453.758362,439.600128,449.679626,1.209038e+10,1.0,17.566573,459.289341,361.315451,59.830419,167.272639,46.126883,399.548564,379.523842,24.799999,0.000000


## Data Split and Save

In [ ]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)

print(len(train))
print(len(trade))

1512
498


In [ ]:
train.to_csv('./train.csv')
trade.to_csv('./trade.csv')

print(f"train.shape: {train.shape}")
print(f"trade.shape: {trade.shape}")

train.shape: (1512, 18)
trade.shape: (498, 18)


# 2. Train, Test, & Make Env
## Design Env

In [ ]:
crypto_dimension = len(processed_full.tic.unique()) # the number of cryptocurrencies
state_space = 1+ 2*crypto_dimension + len(INDICATORS)*crypto_dimension
print(f"Crypto dimension: {crypto_dimension}, State space: {state_space}")

Crypto dimension: 6, State space: 61


We're using stock env of FinRl due to many packaging issues in the crypto env and the similar behaviour and outdome of both on RL terms

In [ ]:
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

buy_cost_list = sell_cost_list = [0.001] * crypto_dimension   # the transaction costs for buying and selling each cryptocurrency
num_crypto_shares = [0] * crypto_dimension

env_kwargs = {
    "hmax": 100,  # maximum number of steps per episode
    "initial_amount": 1000000,  # initial investment amount
    "num_stock_shares": num_crypto_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": crypto_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": crypto_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env() #vectorizing the initialized env
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


## initialize agent & train

In [ ]:
from finrl.agents.stablebaselines3.models import DRLAgent

agent = DRLAgent(env = env_train)

#set the corresponding values to True for the algorithms that you want to use
if_using_sac = False
if_using_ddpg = False
if_using_ppo = True
if_using_a2c = True
if_using_td3 = False

In [ ]:
from stable_baselines3.common.logger import configure
from finrl.config import TRAINED_MODEL_DIR, RESULTS_DIR

model_a2c = agent.get_model("a2c")
if if_using_a2c:
  #set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])    #set ups three types of log (console csv and tensorboard)
  #set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [ ]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

-------------------------------------
| time/                 |           |
|    fps                | 317       |
|    iterations         | 100       |
|    time_elapsed       | 1         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -8.5      |
|    explained_variance | -0.044    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 92.8      |
|    reward             | 6.9659696 |
|    std                | 0.997     |
|    value_loss         | 173       |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 317       |
|    iterations         | 200       |
|    time_elapsed       | 3         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -8.53     |
|    explained_variance | -0.0304   |
|    learning_rate      | 0.0007    |
|    n_updat

In [ ]:
trained_a2c.save(TRAINED_MODEL_DIR + '/agent_a2c') if if_using_a2c else None

## Testing

In [ ]:
crypto_dimension_dimention = len(trade.tic.unique())
state_space = 1+ 2*crypto_dimension + len(INDICATORS)*crypto_dimension
print(f"Trade dimension: {crypto_dimension}, State space: {state_space}")

Trade dimension: 6, State space: 61


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * crypto_dimension   # the transaction costs for buying and selling each cryptocurrency
num_crypto_shares = [0] * crypto_dimension

env_kwargs = {
    "hmax": 100,  # maximum number of steps per episode
    "initial_amount": 1000000,  # initial investment amount
    "num_stock_shares": num_crypto_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": crypto_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": crypto_dimension,
    "reward_scaling": 1e-4
}

e_train_gym = StockTradingEnv(df = trade, turbulence_threshold=70, risk_indicator_col='vix', **env_kwargs)

In [ ]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(model=trained_a2c,
                                               environment = e_train_gym) if if_using_a2c else (None, None)

hit end!


## 3. Backtesing

In [ ]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0]) #sets date as index

result = pd.DataFrame()
if if_using_a2c: result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)

In [ ]:
col_name = []
col_name.append('A2C') if if_using_a2c else None
result.columns = col_name

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure()
plt.plot(result, marker='o')  # Adjust the plotting methoblob:https://colab.research.google.com/1b32bf29-53b0-4ef7-af20-5f784216aa02d as needed

# Save the plot as an image file
plt.savefig('result_plot.png')  # Save in the current working directory
plt.close()

## Mean Variance Optimizatio (MVO)

In [ ]:
#helps us process data into a form for wight calculation

def process_df_for_mvo(df):
    df = df.sort_values(['date', 'tic'], ignore_index=True)[['date','tic', 'close']]
    fst = df
    fst = fst.iloc[0:crypto_dimension, :]
    tic = fst['tic'].tolist()

    mvo = pd.DataFrame()

    for k in range(len(tic)):
     mvo[tic[k]] = 0

    for i in range(df.shape[0]//crypto_dimension):
      n = df
      n = n.iloc[i * crypto_dimension:(i+1) * crypto_dimension, :]
      date = n['date'][i*crypto_dimension]
      mvo.loc[date] = n['close'].tolist()


    return mvo

def crypto_returns_computing(cryptoPrice, rows, columns):
    cryptoReturns = np.zeros((rows-1, columns))
    for j in range(columns):      # j:assets
      for i in range(rows-1):     #i: daily prices
        cryptoReturns[i, j] = (cryptoPrice[i+1, j] - cryptoPrice[i, j])/cryptoPrice[i, j]
    return cryptoReturns

In [ ]:
cryptoData = process_df_for_mvo(train)
tradeData = process_df_for_mvo(trade)

tradeData.to_numpy()

#compute asset returns
arCryptoPrices = np.asarray(cryptoData)
[rows, cols] = arCryptoPrices.shape
arReturns = crypto_returns_computing(arCryptoPrices, rows, cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis=0)
covMatrix = np.cov(arReturns, rowvar=False)

#set percision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print("Mean returns of assets in k-portfolio 1:\n", meanReturns)
print("Variance-covariance matrix of returns:\n", covMatrix)

Mean returns of assets in k-portfolio 1:
 [0.009 0.005 0.006 0.007 0.009 0.004]
Variance-covariance matrix of returns:
 [[0.005 0.004 0.003 0.002 0.004 0.004]
 [0.004 0.004 0.003 0.002 0.003 0.003]
 [0.003 0.003 0.003 0.002 0.003 0.002]
 [0.002 0.002 0.002 0.002 0.002 0.002]
 [0.004 0.003 0.003 0.002 0.004 0.003]
 [0.004 0.003 0.002 0.002 0.003 0.007]]


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covMatrix, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe(risk_free_rate=0)
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(6)])

In [ ]:
lastPrice = np.array([1/p for p in cryptoData.tail(1).to_numpy()[0]])
initialPortfolio = np.multiply(mvo_weights, lastPrice)

portfolio_assets = tradeData @ initialPortfolio
mvo_result = pd.DataFrame(portfolio_assets, columns=['Mean Var'])

mvo_result

,Mean Var
2021-09-01,4.485948e+06
2021-09-02,4.523362e+06
2021-09-03,4.617061e+06
2021-09-07,4.027849e+06
2021-09-08,4.037279e+06
...,...
2021-12-22,3.690883e+06
2021-12-23,3.877258e+06
2021-12-27,3.859129e+06
2021-12-28,3.619112e+06


In [ ]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])

result = pd.DataFrame()
if if_using_a2c: result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)

In [ ]:
col_name = []
col_name.append('A2C') if if_using_a2c else None
result.columns = col_name

In [ ]:
result = pd.merge(result, mvo_result, how='outer', left_index=True, right_index=True)
result

,A2C,Mean Var
date,,
2021-09-01,1.000000e+06,4.485948e+06
2021-09-02,1.008882e+06,4.523362e+06
2021-09-03,1.024425e+06,4.617061e+06
2021-09-07,9.587516e+05,4.027849e+06
2021-09-08,9.440175e+05,4.037279e+06
...,...,...
2021-12-22,9.873289e+05,3.690883e+06
2021-12-23,1.031023e+06,3.877258e+06
2021-12-27,1.028446e+06,3.859129e+06


In [39]:
plt.rcParams["figure.figsize"] = (15, 5)
plt.figure()
plt.plot(result.index, result['A2C'], marker='o', label='a2c')
plt.plot(result.index, result['Mean Var'], marker='o', label='mvo')
plt.legend()  # Add legend based on the labels specified in the plot commands
plt.savefig('combined_result_plot.png')  # Save in the current working directory
plt.close()